## Neural Collaborative Filtering

Neural Collaborative Filtering (NCF) is an algorithm based on deep neural networks to tackle collaborative filtering on the basis of implicit feedback. Since we are using neural networks to find relation between users and items, we can easily scale the solution to large datasets. Thus making this method better than item based collaborative filtering.

NCF works by first representing users and items as vectors in a latent space. These vectors are then used to calculate a score for each user-item pair. The score is then used to predict whether the user will interact with the item. NCF is useful because it can learn non-linear relationships between users and items. This makes it a more powerful model than traditional matrix factorization methods.

Reference: [https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/ncf_deep_dive.ipynb]

### Setting up the environment


In [ ]:
# This is only necessary for colab since it only supports python 3.10, but the library we are using only supports <= 3.9.
# Comment this section if you are running it on your local machine

!sudo rm -rf /usr/local/lib/python3.8/dist-packages/OpenSSL
!sudo rm -rf /usr/local/lib/python3.8/dist-packages/pyOpenSSL-22.1.0.dist-info/

!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.5.2-0-Linux-x86_64.sh

!bash ./Miniconda3-py39_23.5.2-0-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
!pip3 install pyOpenSSL==22.0.0

# Installing the recommenders library.
# Ensure that you have python version <=3.9 when installing this.
!pip install recommenders[examples]

--2024-04-02 09:03:15--  https://repo.anaconda.com/miniconda/Miniconda3-py39_23.5.2-0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93409434 (89M) [application/x-sh]
Saving to: ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.6’

Miniconda3-py39_23. 100%[===================>]  89.08M   168MB/s    in 0.5s    

2024-04-02 09:03:16 (168 MB/s) - ‘Miniconda3-py39_23.5.2-0-Linux-x86_64.sh.6’ saved [93409434/93409434]

PREFIX=/usr/local
Unpacking payload ...
                                                                                                   
Installing base environment...



Preparing transaction: - done
Executing transaction: | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running th

In [1]:
import sys
import os
import shutil

import pandas as pd
import numpy as np

from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_chrono_split, python_stratified_split

from recommenders.models.ncf.dataset import Dataset as NCFDataset

# Importing the NCF model class from the recommenders library
from recommenders.models.ncf.ncf_singlenode import NCF

# importing the evaluation metrics
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items,
                                                     catalog_coverage, distributional_coverage, novelty, diversity, serendipity)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



System version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.4.2


### Loading the Dataset

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_full = pd.read_csv('user_songs_filtered.csv')[['Username', 'track_name', 'artist_name',
                                                    'rank',
                                                    # 'playcount'
                                                    ]]
df_full['track'] = df_full['track_name'] + ' ' + df_full['artist_name']
df_full['itemID'] = df_full.groupby('track').ngroup() + 1
df_full['userID'] = df_full.groupby('Username').ngroup() + 1

df = df_full.copy()
df.rename(columns={'rank': 'rating'}, inplace=True)
# df.rename(columns={'playcount': 'rating'}, inplace=True)
df = df.drop(['track', 'track_name', 'artist_name', 'Username'], axis = 1)

# using a subset of data to reduce runtime to manageable duration, select users who have more than 48 top songs
threshold = 48
df = df[df.groupby('userID')['userID'].transform('size') >= threshold]
df = df[['userID', 'itemID', 'rating']]
df.info()

# df.to_csv('df_NCF.csv', index = False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 94748 entries, 0 to 393058
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   userID  94748 non-null  int64
 1   itemID  94748 non-null  int64
 2   rating  94748 non-null  int64
dtypes: int64(3)
memory usage: 2.9 MB


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/df_NCF.csv')
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94748 entries, 0 to 94747
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rating  94748 non-null  int64
 1   itemID  94748 non-null  int64
 2   userID  94748 non-null  int64
dtypes: int64(3)
memory usage: 2.2 MB


Stratified Split: this is similar to random sampling, but the splits are stratified, for example if the datasets are split by user, the splitting approach will attempt to maintain the same ratio of items used in both training and test splits. The converse is true if splitting by item.


In [5]:
#Split the dataset into 75% train and 25% test

# header = {
#     "col_user": "userID",
#     "col_item": "itemID",
#     "col_rank": "rank",
#     # "col_rank": 'playcount',
#     "col_prediction": "Prediction",
# }


train, test = python_stratified_split(
    df, ratio=0.85,
    #   col_user="userID", col_item="itemID", seed=42
)

# Filtering out users and items in the test set that do not appear in the training set.
# This is done so that we can see if our model has learnt user's previous item interactions and can recommend relevant items.
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

# Creating a test set which only contains the last interaction for each user. Remaining data of the user is used in the train set
leave_one_out_test = test.groupby("userID").last().reset_index()

test.head()

,userID,itemID,rating
232162,5,16117,20
267228,6,6898,40
185573,6,18824,3
219164,10,83514,10
239218,10,66049,4


In [6]:
test.shape

(7775, 3)

In [7]:
# top k items to recommend
TOP_K = 30
# TOP_K = 10

# Model parameters
# Number of iterations during the training process
EPOCHS = 100
# Batch size means how many user-item pairs you want to predict at once
BATCH_SIZE = 256

# Setting seed to remove any stochasticity and reproduce results
SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [8]:
# Writing the data into csv files
train_file = "train.csv"
test_file = "test.csv"
leave_one_out_test_file = "leave_one_out_test.csv"

train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [9]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

INFO:recommenders.models.ncf.dataset:Indexing train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing leave_one_out_test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file leave_one_out_test_full.csv ...
100%|██████████| 1882/1882 [00:36<00:00, 52.03it/s]
INFO:recommenders.models.ncf.dataset:Indexing leave_one_out_test_full.csv ...


### Training the NCF Model

NCF parameters:

`n_users`, number of users. We are one hot encoding our user data. Therefore the input size of the model will be number of users.

`n_items`, number of items. Same logic as `n_users`.

`batch_size`, number of examples you want the model to process at a time. Higher value will consume more memory.

`learning_rate`, this can be thought of as how much you want the model to change after one iteration. Large value will lead to unstability and very small values will take more time to converge.

`n_factors`, which controls the dimension of the latent space. Usually, the quality of the training set predictions grows with as n_factors gets higher.

`layer_sizes`, sizes of input layer (and hidden layers) of MLP, input type is list. We have set it to [64,32,16,8,4] as from training and testing, higher values gave better results.

`n_epochs`, which defines the number of iteration of the SGD procedure. Note that both parameter also affect the training time.

`model_type`, we can train single "MLP", "GMF" or combined model "NCF" by changing the type of model.

[Reference: https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/ncf_deep_dive.ipynb]

In [10]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[64,32,16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

C:\Users\e0559632\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '




In [11]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [26.50s]: train_loss = 0.107274 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [27.63s]: train_loss = 0.041842 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [26.95s]: train_loss = 0.027858 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [26.60s]: train_loss = 0.020979 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [26.83s]: train_loss = 0.018659 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 60 [26.82s]: train_loss = 0.014891 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 70 [27.74s]: train_loss = 0.013361 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 80 [26.69s]: train_loss = 0.011395 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 90 [26.58s]: train_loss = 0.010642 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 100 [26.50s]: train_loss = 0.009772 


Took 2693.3343105 seconds for training.


In [12]:
# save the model
# https://github.com/recommenders-team/recommenders/issues/1735
dir_path = 'NCF_model_trained'
model.save(dir_path)
# # and then while loading depending on the type of your model in this case neumf pass it that dir parameter
# model.load(neumf_dir='dir_path')

### Prediction

After fitting the model, we can call `predict` to get some predictions. `predict` returns an internal object Prediction which can be easily converted back to a dataframe.

In [11]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,6,69610,4.548416e-28
1,6,92294,3.256148e-17
2,10,121378,1.326578e-18
3,10,83514,1.529767e-11
4,10,129097,1.347210e-09


### Generic Evaluation

We remove songs that are already users' top songs in the top k recommendations. To compute ranking metrics, we need predictions on all user, item pairs. We do not want to recommend the same item again to the user.

- Ranking Metrics: These are used to evaluate how relevant recommendations are for users

MAP - It is the average precision for each user normalized over all users.

Normalized Discounted Cumulative Gain (NDCG) - evaluates how well the predicted items for a user are ranked based on relevance

Precision - this measures the proportion of recommended items that are relevant

Recall - this measures the proportion of relevant items that are recommended


- Rating Metrics: These are used to evaluate how accurate a recommender is at predicting ratings that users gave to items

Root Mean Square Error (RMSE) - measure of average error in predicted ratings

R Squared (R2) - essentially how much of the total variation is explained by the model

Mean Absolute Error (MAE) - similar to RMSE but uses absolute value instead of squaring and taking the root of the average

Explained Variance - how much of the variance in the data is explained by the model


- Non accuracy based metrics: These do not compare predictions against ground truth but instead evaluate the following properties of the recommendations

Novelty - measures of how novel recommendation items are by calculating their recommendation frequency among users

Diversity - measures of how different items in a set are with respect to each other

Serendipity - measures of how surprising recommendations are to to a specific user by comparing them to the items that the user has already interacted with

Coverage - measures related to the distribution of items recommended by the system.

In [13]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 159.64166509999995 seconds for prediction.


In [ ]:
# # all for the RAM limitations...
# all_predictions = pd.read_csv("/content/drive/MyDrive/all_predictions_1.csv")
# train = pd.read_csv("/content/drive/MyDrive/train.csv")

# merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
# all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

# all_predictions.to_csv("/content/drive/MyDrive/all_predictions_2.csv", index = False)

In [ ]:
# import pandas as pd
# # importing the evaluation metrics
# from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
#                                                      recall_at_k, get_top_k_items)

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# all_predictions = pd.read_csv("all_predictions_2.csv")
# test = pd.read_csv("test.csv")
# TOP_K = 10

In [15]:
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"Precision: {eval_precision} \n Recall: {eval_recall}")

Precision: 0.019753086419753086 
 Recall: 0.07394472394472394


In [16]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"MAP@K: {eval_map}")

MAP@K: 0.02070399999658074


In [17]:
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"NDCG@K: {eval_ndcg}")

NDCG@K: 0.055243257997956347


In [26]:
# Prepare for diversity based evaluations

# Merge all_predictions with train on userID and itemID
merged_df = pd.merge(all_predictions, train, left_on=['userID', 'itemID'], right_on=['userID', 'itemID'], how='outer')

# Filter out the rows where train.rating is null
top_all = merged_df[merged_df['rating'].isnull()]
top_all = top_all[['userID', 'itemID', 'prediction']]
print(top_all.shape[0])

# Sort top_all DataFrame by 'prediction' column within each 'userID' group in descending order
top_all_sorted = top_all.sort_values(by=['userID', 'prediction'], ascending=[True, False])

# Group by 'userID' and take the top_k items for each group
top_k_reco = top_all_sorted.groupby('userID').head(TOP_K)
print(top_k_reco.shape[0])

25320007
31350


In [32]:
eval_diversity = diversity(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"Diversity: {eval_diversity}")

eval_novelty = novelty(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"Novelty: {eval_novelty}")

eval_distributional_coverage = distributional_coverage(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"distributional_coverage: {eval_distributional_coverage}")

eval_catalog_coverage = catalog_coverage(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"catalog_coverage: {eval_catalog_coverage}")

eval_serendipity = serendipity(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"serendipity: {eval_serendipity}")

Diversity: 0.9540511859603287
Novelty: 12.92741773007934
distributional_coverage: 11.544841944811079
catalog_coverage: 0.23628796307743025
serendipity: 0.9828262905371942


In [28]:
eval_novelty = novelty(train, top_k_reco, col_user='userID', col_item='itemID')
print(f"Novelty: {eval_novelty}")


Novelty: 12.92741773007934


### Summary of Ranking Metrics

<center>

|Metric|Range|Selection criteria|Limitation|
|------|-------------------------------|---------|----------|
|Precision|$\geq 0$ and $\leq 1$|Higher the better.|Only for hits in recommendations.|
|Recall|$\geq 0$ and $\leq 1$|Higher the better.|Only for hits in the ground truth.|
|NDCG|$\geq 0$ and $\leq 1$|Higher the better.|Does not penalize for bad/missing items, and does not perform for several equally good items.|
|MAP|$\geq 0$ and $\leq 1$|Higher the better.|Depend on variable distributions.|

</center>

### "Leave-one-out" Evaluation

For each item in test data, we randomly samples 100 items that are not interacted by the user, ranking the test item among the 101 items (1 positive item and 100 negative items). The performance of a ranked list is judged by Hit Ratio (HR) and Normalized Discounted Cumulative Gain (NDCG). Finally, we average the values of those ranked lists to obtain the overall HR and NDCG on test data.

We truncated the ranked list at 10 for both metrics. As such, the HR intuitively measures whether the test item is present on the top-10 list, and the NDCG accounts for the position of the hit by assigning higher scores to hits at top ranks.

In [ ]:
k = TOP_K

ndcgs = []
hit_ratio = []

for b in data.test_loader():
    user_input, item_input, labels = b
    output = model.predict(user_input, item_input, is_list=True)

    output = np.squeeze(output)
    rank = sum(output >= output[0])
    if rank <= k:
        ndcgs.append(1 / np.log(rank + 1))
        hit_ratio.append(1)
    else:
        ndcgs.append(0)
        hit_ratio.append(0)

eval_ndcg = np.mean(ndcgs)
eval_hr = np.mean(hit_ratio)

print("HR:\t%f" % eval_hr)
print("NDCG:\t%f" % eval_ndcg)

HR:	0.506507
NDCG:	0.444740
